# Tutorial 6: Coherent Shifts
## Natural Transformations Between Functors

---

### A Second Manuscript

*To the research assistant:*

*In Year 897, Tessery Vold completed her second major work: "On the Coherence of Shifts." Where her first work established passage diagrams (categories) and preservation maps (functors), this manuscript addressed a deeper question: how do different preservation maps relate to each other?*

*Consider the challenge of translating documents between the Capital Archive and the Western Archive. There is not one translation—there are many. The formal translation used by archivists. The colloquial translation used by traveling scholars. The historical translation based on old conventions.*

*Each translation is a functor. But the translations must "cohere"—they cannot assign arbitrary meanings independently. If two passages compose in the Capital system, their translations must compose in the Western system. Vold realized that the relationship between translations is itself structured.*

*She called these relationships "coherent shifts"—what modern category theorists call natural transformations.*

*Your task: Analyze the regional translations data to understand how different functors relate. Determine when a shift between functors is coherent—and when it fails to be.*

—*Archive Review Committee, Year 934*

---

## What You Will Learn

In this tutorial, you will learn to:

1. Understand **natural transformations** as morphisms between functors
2. Recognize the **naturality square** (the "prism diagram")
3. Test whether a transformation is natural (coherent)
4. Understand **components** of a natural transformation
5. See the connection to real-world translation systems

By the end, you will understand:
- Why natural transformations are "the right" notion of morphism between functors
- The coherence condition that makes transformations "natural"
- How this relates to attention mechanisms and representation learning

---

## The Prism Diagram

Given two functors F, G: C → D, a **natural transformation** η: F ⇒ G is:

- For each object X in C, a morphism η_X: F(X) → G(X) in D
- Such that for every morphism f: X → Y in C, the following diagram commutes:

```
        F(f)
F(X) -------→ F(Y)
 |             |
 | η_X         | η_Y
 ↓             ↓
G(X) -------→ G(Y)
        G(f)
```

This means: η_Y ∘ F(f) = G(f) ∘ η_X

### Vold's Interpretation

> *"A coherent shift does not merely translate each passage independently. It translates in a way that respects composition. If a creature passes from A to B to C, the shift must carry that entire journey coherently—not just the starting point, not just the ending point, but the structure of the passage itself."*
> — Tessery Vold, "On the Coherence of Shifts," Year 897

---

## The Currency Analogy

Imagine you have two ways to convert currency:
- F: Convert prices directly (1 league = 2 marks)
- G: Convert prices via a third currency (1 league = 5 thalers = 2 marks)

A natural transformation η: F ⇒ G would be a way to "shift" from the direct conversion to the indirect conversion, such that the shift respects price differences.

If item A costs 10 leagues and item B costs 15 leagues, then:
- F(A) = 20 marks, F(B) = 30 marks (difference: 10 marks)
- G(A) = 20 marks, G(B) = 30 marks (difference: 10 marks)

The shift η must preserve the relationship between A and B's prices, not just their absolute values.

---

## Part 1: Loading the Translation Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from collections import defaultdict

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('deep')

print("Libraries loaded. Ready to study coherent shifts.")

In [ ]:
# Load the regional translations data
BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

translations = pd.read_csv(BASE_URL + "regional_translations.csv")

print(f"Translations loaded: {len(translations)} records")
print(f"\nColumns: {list(translations.columns)}")

In [ ]:
# View sample translations
translations.head(10)

Each row represents a **component** of a functor—a mapping from one system to another.

Key columns:
- `source_system` / `target_system`: The source and target categories
- `source_term` / `target_term`: How a term is translated
- `preserves_structure`: Whether this mapping respects composition
- `translation_type`: What kind of translation (category_mapping, terminology, etc.)

---

## Part 2: Identifying Functors

A functor is a collection of mappings from one system to another. Let's identify the functors present in the data.

In [ ]:
# What source/target system pairs exist?
functor_pairs = translations.groupby(['source_system', 'target_system']).size().reset_index(name='component_count')

print("Potential functors (system → system mappings):")
print("=" * 60)
for _, row in functor_pairs.sort_values('component_count', ascending=False).iterrows():
    print(f"  {row['source_system']:25} → {row['target_system']:25} ({row['component_count']} components)")

In [ ]:
# Visualize the functor network
G_functors = nx.DiGraph()

for _, row in functor_pairs.iterrows():
    G_functors.add_edge(row['source_system'], row['target_system'], 
                        weight=row['component_count'])

fig, ax = plt.subplots(figsize=(14, 10))

pos = nx.spring_layout(G_functors, k=3, iterations=50, seed=42)

# Draw nodes
nx.draw_networkx_nodes(G_functors, pos, node_size=2500, node_color='lightblue', ax=ax)
nx.draw_networkx_labels(G_functors, pos, font_size=8, ax=ax)

# Draw edges with varying thickness
edges = G_functors.edges(data=True)
weights = [e[2]['weight'] for e in edges]
nx.draw_networkx_edges(G_functors, pos, edge_color='gray', 
                        arrows=True, arrowsize=20,
                        width=[w/2 for w in weights],
                        connectionstyle='arc3,rad=0.1', ax=ax)

ax.set_title('Functor Network: Translation Systems\n(Edge weight = number of term mappings)', fontsize=12)
ax.axis('off')
plt.tight_layout()
plt.show()

The network shows how translation functors connect different systems. Some systems have multiple functors between them (e.g., capital → western via different paths).

---

## Part 3: Structure Preservation

Not all mappings are functors—only those that **preserve structure** (composition and identity). Let's analyze which translations meet this requirement.

In [ ]:
# Which translations preserve structure?
preservation = translations.groupby(['source_system', 'target_system', 'preserves_structure']).size().unstack(fill_value=0)

print("Structure preservation by functor:")
print("=" * 60)
print(preservation)

In [ ]:
# Find fully structure-preserving functors (all components preserve structure)
full_functors = translations.groupby(['source_system', 'target_system']).agg({
    'preserves_structure': lambda x: x.all(),
    'translation_id': 'count'
}).reset_index()

full_functors.columns = ['source_system', 'target_system', 'is_true_functor', 'component_count']

print("Fully structure-preserving functors:")
print(full_functors[full_functors['is_true_functor']])

In [ ]:
# Example of a structure-breaking translation
non_preserving = translations[translations['preserves_structure'] == False]

print("Examples of structure-breaking translations:")
print("=" * 60)
print(non_preserving[['source_system', 'target_system', 'source_term', 'target_term', 'notes']])

Notice the pattern: translations that break structure often involve:
- Functional rather than structural mappings (trade routes vs. geography)
- Imprecise historical categories
- Metaphorical/analogical rather than literal translations

These are NOT valid functors because they don't respect the categorical structure.

---

## Part 4: Components of a Natural Transformation

Now let's look at how we might have multiple functors between the same categories, and how they relate.

The **capital_archive → western_archive** translation has a clear inverse: **western_archive → capital_archive**. Together, these form a pair of functors. But there could also be *different* translations—historical vs. modern, formal vs. informal.

In [ ]:
# Focus on the archive translation functor
archive_functor = translations[
    (translations['source_system'] == 'capital_archive') & 
    (translations['target_system'] == 'western_archive')
]

print("Components of F: capital_archive → western_archive")
print("=" * 60)
print(archive_functor[['source_term', 'target_term', 'preserves_structure', 'notes']])

In [ ]:
# And the inverse functor
archive_inverse = translations[
    (translations['source_system'] == 'western_archive') & 
    (translations['target_system'] == 'capital_archive')
]

print("Components of G: western_archive → capital_archive")
print("=" * 60)
print(archive_inverse[['source_term', 'target_term', 'preserves_structure', 'notes']])

The archive translations form an **adjunction**—F and G are mutually inverse (up to isomorphism). The natural transformation between them is the identity.

More interesting is when we have *different* functors with the same source and target that are NOT inverses.

---

## Part 5: Building a Natural Transformation

Let's construct a concrete example. Consider:
- F: Vold's terminology translation (mathematical_notation → natural_language)
- G: A hypothetical "formal" translation (same source/target)

A natural transformation η: F ⇒ G would be a coherent way to shift from Vold's informal terms to formal terms.

In [ ]:
# Vold's terminology translations
vold_terminology = translations[
    (translations['source_system'] == 'mathematical_notation') & 
    (translations['target_system'] == 'natural_language')
]

print("F = Vold's terminology functor:")
print("=" * 50)
for _, row in vold_terminology.iterrows():
    print(f"  F({row['source_term']}) = {row['target_term']}")

In [ ]:
# Define a hypothetical "formal" functor G
# This would map the same terms but to more academic language

formal_translations = {
    'morphism': 'arrow',
    'object': 'element', 
    'functor': 'structure-preserving map',
    'natural_transformation': 'functorial transformation',
    'hom_functor': 'representable presheaf'
}

print("G = Formal academic functor:")
print("=" * 50)
for source, target in formal_translations.items():
    print(f"  G({source}) = {target}")

In [ ]:
# A natural transformation η: F ⇒ G assigns to each object X a morphism η_X: F(X) → G(X)
# In the language category, morphisms are substring inclusions or phrase relationships

# For simplicity, let's think of η_X as a mapping from Vold's term to the formal term

vold_terms = dict(zip(vold_terminology['source_term'], vold_terminology['target_term']))

print("Natural transformation η: Vold → Formal")
print("=" * 50)
print("For each mathematical term X:")
for source in vold_terms:
    if source in formal_translations:
        vold_target = vold_terms[source]
        formal_target = formal_translations[source]
        print(f"\n  X = {source}")
        print(f"  F(X) = {vold_target}")
        print(f"  G(X) = {formal_target}")
        print(f"  η_X: {vold_target} → {formal_target}")

Each component η_X shows how to shift from Vold's accessible term to the formal academic term.

But is this transformation **natural**? We need to check the naturality condition.

---

## Part 6: The Naturality Condition

For η: F ⇒ G to be natural, the following must commute for every morphism f: X → Y:

```
        F(f)
F(X) -------→ F(Y)
 |             |
 | η_X         | η_Y
 ↓             ↓
G(X) -------→ G(Y)
        G(f)
```

This means: going F(X) → F(Y) → G(Y) equals going F(X) → G(X) → G(Y).

In [ ]:
# Let's check naturality with a concrete example from the taxonomy translations

# Consider the morphism: predators → apex_predators (in capital_taxonomy)
# This is a classification hierarchy morphism

# Functor F: capital_taxonomy → western_taxonomy
capital_western = translations[
    (translations['source_system'] == 'capital_taxonomy') & 
    (translations['target_system'] == 'western_taxonomy')
]

print("Capital → Western taxonomy translations:")
print(capital_western[['source_term', 'target_term', 'preserves_structure']])

In [ ]:
# Functor G: capital_taxonomy → northern_taxonomy
capital_northern = translations[
    (translations['source_system'] == 'capital_taxonomy') & 
    (translations['target_system'] == 'northern_taxonomy')
]

print("Capital → Northern taxonomy translations:")
print(capital_northern[['source_term', 'target_term', 'preserves_structure']])

In [ ]:
# To define a natural transformation η: F ⇒ G, we'd need:
# η: western_taxonomy → northern_taxonomy
# That is coherent with F and G

# Let's see if such a translation exists
western_northern = translations[
    (translations['source_system'] == 'western_taxonomy') & 
    (translations['target_system'] == 'northern_taxonomy')
]

if len(western_northern) > 0:
    print("Western → Northern taxonomy translations exist!")
    print(western_northern[['source_term', 'target_term']])
else:
    print("No direct western → northern translation found in data.")
    print("This means η must be constructed from existing components.")

In [ ]:
# Demonstrate naturality check with available data
# We can construct η from F and G by composition:
# If F: capital → western and G: capital → northern
# Then η: F ⇒ G means for each term X in capital:
#   F(X) → G(X), i.e., western_term → northern_term

F_dict = dict(zip(capital_western['source_term'], capital_western['target_term']))
G_dict = dict(zip(capital_northern['source_term'], capital_northern['target_term']))

print("Constructing η: F ⇒ G")
print("=" * 50)
print("\nFor each capital term X that both F and G translate:")

common_terms = set(F_dict.keys()) & set(G_dict.keys())
eta_components = {}

for term in sorted(common_terms):
    f_term = F_dict[term]
    g_term = G_dict[term]
    eta_components[f_term] = g_term
    print(f"\n  X = {term}")
    print(f"  F(X) = {f_term}")
    print(f"  G(X) = {g_term}")
    print(f"  η_X: {f_term} → {g_term}")

We've constructed the components of η. But is η natural? We need to check that the naturality squares commute.

---

## Part 7: Visualizing the Naturality Square

In [ ]:
# Visualize a naturality square
# Example: morphism f: predators → aquatic_fauna (if it existed in the category)

fig, ax = plt.subplots(figsize=(12, 8))

# Positions for the square
positions = {
    'F(predators)': (0, 1),
    'F(apex)': (2, 1),
    'G(predators)': (0, 0),
    'G(apex)': (2, 0)
}

# Create graph
G_square = nx.DiGraph()
G_square.add_nodes_from(positions.keys())

# Add edges
G_square.add_edge('F(predators)', 'F(apex)', label='F(f)')
G_square.add_edge('G(predators)', 'G(apex)', label='G(f)')
G_square.add_edge('F(predators)', 'G(predators)', label='η_pred')
G_square.add_edge('F(apex)', 'G(apex)', label='η_apex')

# Draw
node_labels = {
    'F(predators)': f"F(predators)\n= 'hunters'",
    'F(apex)': f"F(apex)\n= (derived)",
    'G(predators)': f"G(predators)\n= 'meat_takers'",
    'G(apex)': f"G(apex)\n= (derived)"
}

nx.draw_networkx_nodes(G_square, positions, node_size=4000, node_color='lightblue', ax=ax)
nx.draw_networkx_labels(G_square, positions, labels=node_labels, font_size=9, ax=ax)
nx.draw_networkx_edges(G_square, positions, edge_color='gray', arrows=True, 
                        arrowsize=20, connectionstyle='arc3,rad=0.0', ax=ax)

edge_labels = {('F(predators)', 'F(apex)'): 'F(f)',
               ('G(predators)', 'G(apex)'): 'G(f)',
               ('F(predators)', 'G(predators)'): 'η_predators',
               ('F(apex)', 'G(apex)'): 'η_apex'}
nx.draw_networkx_edge_labels(G_square, positions, edge_labels=edge_labels, 
                              font_size=10, ax=ax)

ax.set_title('The Naturality Square\n(Must commute for η to be natural)', fontsize=12)
ax.text(1, 0.5, 'This square\nmust commute!', fontsize=11, ha='center', 
        style='italic', color='darkred')
ax.axis('off')
ax.set_xlim(-0.8, 2.8)
ax.set_ylim(-0.5, 1.5)
plt.tight_layout()
plt.show()

The naturality condition says:
- Going across (F(f)) then down (η_apex) 
- Equals going down (η_predators) then across (G(f))

If this fails, η is NOT a natural transformation—it's not a coherent shift.

---

## Part 8: When Coherence Fails

Let's examine cases where translations FAIL to be coherent shifts.

In [ ]:
# Cross-framework translations often fail to be natural
cross_framework = translations[translations['translation_type'] == 'cross_framework']

print("Cross-framework translations:")
print("=" * 60)
print(cross_framework[['source_system', 'target_system', 'source_term', 'target_term', 'preserves_structure', 'notes']])

In [ ]:
# The Vold-Pol debate: partial overlap, not full structure preservation
vold_pol = translations[
    ((translations['source_system'] == 'vold_terminology') & (translations['target_system'] == 'pol_terminology')) |
    ((translations['source_system'] == 'pol_terminology') & (translations['target_system'] == 'vold_terminology'))
]

print("Vold ↔ Pol terminology translations:")
print("=" * 60)
for _, row in vold_pol.iterrows():
    status = "✓" if row['preserves_structure'] else "✗"
    print(f"  {status} {row['source_term']} → {row['target_term']}")
    print(f"      Notes: {row['notes']}")

The Vold-Pol translations are marked as "partial conceptual overlap" and "contested." These are NOT natural transformations because:

1. The concepts don't map cleanly (passage ≠ form_reading)
2. The structure of relationships in Vold's system differs from Pol's
3. There's no coherent way to shift between them while preserving composition

This is why the Year 895 debate remained "unresolved"—there was no natural transformation between their frameworks.

---

## Part 9: The ML Connection

Natural transformations appear throughout machine learning:

### 1. Attention Mechanisms
In transformers, attention maps between different representations. The key-query-value mechanism defines a natural transformation between embedding spaces.

### 2. Layer Normalization
Normalizing across layers can be viewed as a natural transformation that coherently shifts representations.

### 3. Transfer Learning
Fine-tuning a pretrained model is (ideally) a natural transformation: the shift from pretrained to fine-tuned representations should preserve the structure of the original representation.

In [ ]:
# Demonstrate with a simple numerical example
# Two functors F, G that embed words into vectors

# F: word → one-hot encoding
# G: word → dense embedding

vocabulary = ['passage', 'object', 'morphism', 'functor', 'category']

# F: one-hot
F_embeddings = np.eye(len(vocabulary))

# G: random dense embedding (pretend this is learned)
np.random.seed(42)
G_embeddings = np.random.randn(len(vocabulary), 3)  # 3-dimensional

print("F (one-hot) embeddings:")
for i, word in enumerate(vocabulary):
    print(f"  F({word}) = {F_embeddings[i]}")

print("\nG (dense) embeddings:")
for i, word in enumerate(vocabulary):
    print(f"  G({word}) = {G_embeddings[i].round(2)}")

In [ ]:
# A natural transformation η: F ⇒ G would be a matrix W such that:
# η_word: F(word) → G(word)
# W @ F(word) = G(word)

# Since F is one-hot, W = G_embeddings.T gives us exactly this
W = G_embeddings.T  # Shape: (3, 5)

print("Natural transformation η as weight matrix W:")
print(f"Shape: {W.shape}")
print(f"\nW =\n{W.round(2)}")

# Verify: W @ F(word) = G(word)
print("\nVerification:")
for i, word in enumerate(vocabulary[:3]):
    result = W @ F_embeddings[i]
    expected = G_embeddings[i]
    print(f"  η({word}): W @ F({word}) = {result.round(2)}")
    print(f"           G({word})       = {expected.round(2)}")
    print(f"           Match: {np.allclose(result, expected)}")

The weight matrix W is a natural transformation from the one-hot functor to the dense embedding functor. This is how neural network layers work—they define natural transformations between representation spaces.

---

## Exercises

### Exercise 1: Inverse Transformations

The archive functors F: capital → western and G: western → capital are inverses. Show that the identity transformation id: F ∘ G ⇒ id is natural.

In [ ]:
# Your code here
# Hint: Compose the translations and check if they return to the original

### Exercise 2: Horizontal Composition

If we have η: F ⇒ G and θ: H ⇒ K, where G and H are composable, then we can form the horizontal composite θ ∘ η. Find such a pair in the translation data and construct their composite.

In [ ]:
# Your code here
# Hint: Look for chains like capital → western → northern

### Exercise 3: Confidence as Weight

The translations have a `confidence` column. Could confidence levels define a "weighted" natural transformation? Explore how confidence varies across different translation types.

In [ ]:
# Your code here
# Hint: Group by translation_type and analyze confidence distribution

### Exercise 4: Drawing Naturality Squares

Choose two functors from the data and draw at least two naturality squares. Determine whether they commute based on the available information.

In [ ]:
# Your code here
# Hint: Use networkx to visualize the squares

---

## Discussion Questions

1. Vold insisted that translations must be "coherent"—they must respect composition. But in practice, human translation is often context-dependent and inconsistent. Does this mean human translation is not a natural transformation?

2. The Vold-Pol debate failed to produce a natural transformation between their frameworks. What does this say about the relationship between Relational Philosophy and Form Library philosophy?

3. In deep learning, we often "fine-tune" a pretrained model. What conditions would make fine-tuning a natural transformation? When might it fail to be natural?

---

## Summary

In this tutorial, you learned:

| Concept | What You Learned |
|---------|------------------|
| Natural transformation | A morphism between functors: η: F ⇒ G |
| Components | η assigns to each object X a morphism η_X: F(X) → G(X) |
| Naturality condition | η_Y ∘ F(f) = G(f) ∘ η_X for all morphisms f |
| Coherent shift | Vold's term for natural transformation |
| Non-natural mappings | Translations that fail to preserve structure |

| Skill | Code Pattern |
|-------|--------------|
| Identify functors | Group translations by source/target system |
| Check structure preservation | Filter by `preserves_structure` column |
| Construct transformation components | Map source_term → target_term |
| Visualize naturality squares | Use networkx for diagrams |

---

## Next Tutorial

In **Tutorial 7: The Probing Theorem**, you will learn the Yoneda Lemma—the crown jewel of category theory:

- Nat(Hom(-, X), F) ≅ F(X)
- "Objects are determined by how they're probed"
- Why representable functors form a "basis"
- The connection to word embeddings and distributional semantics

> *"The Probing Theorem says: if you know all the natural transformations from the probing pattern to any passage pattern, you know the pattern itself. The object is not the thing—it is the web of coherent relationships."*
> — Tessery Vold, "The Probing Theorem," Year 892

---

## Credits

**Source Material:** Tai-Danae Bradley, "Category Theory and Language Models" (Cartesian Cafe)

**Densworld Integration:** The Relational Foundations course applies categorical concepts through the framework of Tessery Vold.

**Learn more:** [buildLittleWorlds](https://github.com/buildLittleWorlds)

---

> *"A coherent shift transforms not by brute force, but by harmony. Each component shifts in a way that knows the others. This is what separates mere relabeling from true translation."*
> — Tessery Vold, "On the Coherence of Shifts," Year 897